In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time

headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}

In [3]:

main_webUrl = "https://www.rumah123.com/jual/jawa-timur/rumah/?page=1"

req_main = requests.get(main_webUrl, headers=headers)

In [4]:
main_html = BeautifulSoup(req_main.content, "html5lib")

In [5]:
max_page = main_html.find_all("li", attrs={"class": "ui-molecule-paginate__item"})
max_page = int(max_page[-1].text)
max_page

2500

In [6]:
dataframe = {}
dataframe["Nama"] = []
dataframe["Harga"] = []

In [ ]:
for i in range(1, 200):
    webUrl = f"https://www.rumah123.com/jual/jawa-timur/rumah/?page={i}"
    req = requests.get(webUrl, headers=headers)
    time.sleep(3)
    
    html = BeautifulSoup(req.content, "html5lib")
    
    link_sections = html.find_all("div", attrs={"class" : "intersection-card-container"})

    for link_sec in link_sections:
        link = link_sec.find("h2").parent["href"]
        
        if "perumahan-baru" in link:
            continue
        try:
            sub_url = requests.get("https://www.rumah123.com" + link, headers=headers)
            sub_html = BeautifulSoup(sub_url.content, "html5lib")
            time.sleep(2)

            title = sub_html.find("h1", attrs={"class" :"text-gray-800 text mb-5"})
            dataframe["Nama"].append(title.text)
            
            specific_div = sub_html.find("div", attrs={"class" : "w-full md:w-[750px] self-stretch"})
            harga = specific_div.find("span")
            dataframe["Harga"].append(harga.text)

            property_div = sub_html.find("div",attrs={"id" : "property-information"})

            propertyes = [[prob.text,prob.nextSibling.text] for prob in property_div.find_all("p", attrs={"class":"w-32"})]
        except:
            continue
        
        print(link)
        for prop in propertyes:
            prop_key = prop[0]
            prop_value = prop[1]
            
            if not prop_key in dataframe.keys():
                dataframe[prop_key] = []
            
            if prop_key in dataframe.keys():
                dataframe[prop_key].append(prop_value)
            
        print("")


/properti/sidoarjo/hos19398804/

/properti/surabaya/hos19398391/

/properti/malang/hos19257656/

/properti/surabaya/hos19235049/

/properti/surabaya/hos17682414/

/properti/surabaya/hos19386192/

/properti/malang/hos19274306/

/properti/surabaya/hos18860185/

/properti/surabaya/hos19138161/

/properti/surabaya/hos19128533/

/properti/surabaya/hos17192405/

/properti/surabaya/hos12474034/

/properti/surabaya/hos18860892/

/properti/surabaya/hos18917381/

/properti/surabaya/hos19223003/

/properti/surabaya/hos19373162/

/properti/surabaya/hos17265811/

/properti/malang/hos17148877/

/properti/malang/hos16504192/

/properti/surabaya/hos17004192/

/properti/surabaya/hos19388604/

/properti/surabaya/hos19097326/

/properti/surabaya/hos19398925/

/properti/gresik/hos14353967/

/properti/sidoarjo/hos17098154/

/properti/surabaya/hos16354489/

/properti/surabaya/hos16674348/

/properti/surabaya/hos19286217/

/properti/surabaya/hos13469899/

/properti/surabaya/hos14178360/

/properti/surabaya/h

In [ ]:
rumah123_df = pd.DataFrame.from_dict(dataframe, orient='index').T
rumah123_df = rumah123_df.fillna(0)
rumah123_df.to_json("Provinsi_JawaTimur.json", orient="records")

In [9]:
len(rumah123_df)

354